The following code snippet is authored by:<br>
- Markus Fath https://github.com/fath-markus

This notebook shows how you can generate embedding vectors from 
* a (larger) number of documents/texts 
* stored in a HANA table 
* using generative-ai-hub-sdk
* to access AI core services.

There are four atomic functions
* "prepare_table" adds a vector column to your table
* "read_documents" pulls a batch of texts from your table
* "embed_documents" sends one or multiple batches of text to an embedding function hosted on BTP
* "store_documents" writes the embedding vectors to your table

and an orchestrating function *"read_embed_store_documents"*, which runs the last three atomic functions in a loop.

Prerequisites:
- generative-ai-hub-sdk (1.2.0)
- openAI ada deployment on AI Core in your BTP tenant

See:<br>
https://pypi.org/project/generative-ai-hub-sdk/
https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/create-deployment-for-generative-ai-model-in-sap-ai-core


In [1]:
# Connect to HANA using hana-ml
from hana_ml.dataframe import ConnectionContext
# cc = ConnectionContext(userkey='GR3', encrypt=True)
cc= ConnectionContext(
    address='[somehost].hanacloud.ondemand.com', 
    port='443', 
    user='[your user]', 
    password='[your password]', 
    encrypt=True
    )
connection = cc.connection

ModuleNotFoundError: No module named 'shapely'


In [2]:
import time
from datetime import datetime

from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

In [3]:
def prepare_table (schema_name:str, table_name:str, text_column:str):
    """
        Adds a REAL_VECTOR column to a table.
        @param schema_name <str>: Name of HANA schema to load vectors into
        @param table_name <str>: Name of HANA table to load vectors into
        @param text_column <str>: Name of text column of the HANA table
    """
    now = str(datetime.now())
    try:
        curr = None
        curr = cc.connection.cursor()
        sql_command = f'''ALTER TABLE "{schema_name}"."{table_name}" 
            ADD ("{text_column}_VECTOR" REAL_VECTOR(1536) COMMENT 'embedding vector, generated from column {text_column} using openAI ada on {now}')'''
        curr.execute(sql_command)
        curr.close()
    except Exception as exp:
        print(f"Could not alter table {table_name} due to {exp}")
    finally:
        if curr != None:
            curr.close()
            pass
        pass

In [4]:
def read_documents(schema_name:str, table_name:str, key_column:str, text_column:str, batch_size_read:int = 300):
    """
        Read a batch of documents from a table, for which the vector column is NULL.
        @param schema_name <str>: Name of HANA schema to load vectors into
        @param table_name <str>: Name of HANA table to load vectors into
        @param key_column <str>: Name of key column of the HANA table
        @param text_column <str>: Name of text column of the HANA table
        @param batch_size_read <int>: Number of documents to read in each iteration
        @return pandas df
    """
    sql_command = f''' SELECT "{key_column}", "{text_column}" FROM "{schema_name}"."{table_name}" 
        WHERE "{text_column}_VECTOR" IS NULL AND "{text_column}" IS NOT NULL AND LENGTH("{text_column}") > 2 
        LIMIT {batch_size_read} '''
    hdf = cc.sql(sql_command)
    return hdf.collect()

In [5]:
def embed_documents(df, key_column:str, text_column:str, batch_size_embed:int = 300):
    """
        Function to embed documents from given dataframe.
        @param df<pandas df>: dataframe with key and text
        @param key_column <str>: Name of key column of the HANA table
        @param text_column <str>: Name of text column of the HANA table
        @param batch_size_embed <int>: Number of documents for each embedding batch
    """
    # Extract documents and corresponding keys from df
    docs = [(row[text_column], row[key_column]) for _, row in df.iterrows()]

    # Truncate each text s.t. token length <= 8000... 1tok = 4 chars... 8000 tokens < 30k chars
    doc_list = [e[0][:30000] for e in docs]

    # Create the embeddings, pass batch_size and retry options to client
    embedding = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002', chunk_size=batch_size_embed, max_retries=10)
    return embedding.embed_documents(doc_list)


In [6]:
def store_vectors(schema_name:str, table_name:str, key_column:str, text_column:str, df, vector_list):
    """
        Function to load vector embeddings into given HANA table.
        @param schema_name <str>: Name of HANA schema to load vectors into
        @param table_name <str>: Name of HANA table to load vectors into
        @param key_column <str>: Name of key column of the HANA table
        @param text_column <str>: Name of text column of the HANA table
        @param df<pandas df>: dataframe with key and text
        @param vector_list <list<list<float>>>: Document vectors
    """
    docs = [(row[text_column], row[key_column]) for _, row in df.iterrows()]
    # Prepare list of vectors and associated keys
    rows = [(str(e), docs[idx][1]) for idx, e in enumerate(vector_list)]

    # Load vectors in HANA table in transactional mode
    cc.connection.setautocommit(False)
    cursor = None
    try:
        cursor = cc.connection.cursor()
        sql_command = f'''UPDATE "{schema_name}"."{table_name}" SET "{text_column}_VECTOR" = TO_REAL_VECTOR(?) WHERE "{key_column}" = ?'''
        cursor.executemany(sql_command, rows)
        cc.connection.commit()
        print(f"Docs inserted: " + str(len(rows)))
    except Exception as e:
        cc.connection.rollback()
        print("An error occurred:", e)
    finally:
        if cursor != None:
            cursor.close()
            pass
        pass

    cc.connection.setautocommit(True)
    pass

In [17]:
# prepare_table('GRAPH_USER', 'TEST', 'SUMMARY');
# df_docs = read_documents('GRAPH_USER', 'TEST', 'ID', 'SUMMARY', 5)
# vector_list = embed_documents(df_docs, 'ID', 'SUMMARY', 2)
# store_vectors('GRAPH_USER', 'TEST', 'ID', 'SUMMARY', df_docs, vector_list)

Docs inserted: 5


In [18]:
def read_embed_store_documents(schema_name:str, table_name:str, key_column:str, text_column:str, batch_size_read:int, batch_size_embed:int):
    """
        Function to generate vector embeddings for text stored in a HANA table.
        @param schema_name <str>: schema of the HANA table
        @param table_name <str>: name of the HANA table
        @param key_column <str>: key column of the HANA table
        @param text_column <str>: name of the column in which the text is stored
        @param batch_size_read <int>: Number of documents to read in each iteration
        @param batch_size_embed <int>: Number of documents for each embedding batch
    """
    try:
        prepare_table(schema_name=schema_name, table_name=table_name, text_column=text_column)
    except:
        pass
    while True:
        vector_list = []
        number_of_new_docs = 0
        df_docs = read_documents(schema_name=schema_name, table_name=table_name, key_column=key_column, text_column=text_column, batch_size_read=batch_size_read)
        number_of_new_docs = len(df_docs)
        if number_of_new_docs == 0:
            print('All docs embedded.')
            break
        else:
            print('Fetched {n} new docs.'.format(n=number_of_new_docs))
            try:
                print('Embedding {n} documents, using batch size {batch_size_embed}...'.format(n=number_of_new_docs, batch_size_embed=batch_size_embed))
                vector_list = embed_documents(df=df_docs, key_column=key_column, text_column=text_column, batch_size_embed=batch_size_embed)
                print('Done. Storing vectors in HANA...')
                store_vectors(schema_name=schema_name, table_name=table_name, key_column=key_column, text_column=text_column, df=df_docs, vector_list=vector_list)
                print('Done.')
            finally:
                pass
    print('Done')

In [19]:
read_embed_store_documents(
    schema_name='GRAPH_USER', 
    table_name='TEST', 
    key_column='ID', 
    text_column='SUMMARY', 
    batch_size_read=200,
    batch_size_embed=100
    )

Fetched 200 new docs.
Embedding 200 documents, using batch size 100...
Done. Storing vectors in HANA...
Docs inserted: 200
Done.
Fetched 200 new docs.
Embedding 200 documents, using batch size 100...
Done. Storing vectors in HANA...
Docs inserted: 200
Done.
All docs embedded.
Done


In [20]:
# lets try a search
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = embedding.embed_query(query)
    sql = '''SELECT TOP {k} "ID", "SUMMARY", "{metric}"("SUMMARY_VECTOR", TO_REAL_VECTOR('{qv}')) AS SIM
        FROM "GRAPH_USER"."TEST"
        ORDER BY "SIM" {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    return df_context

In [21]:
query = 'taxi luggage mexico motion sickness mosters nasa'
df = run_vector_search(query = query, k = 10)
df

,ID,SUMMARY,SIM
0,3041353,Magic Journeys looked at the world through the...,0.785842
1,11378430,Financial disaster looms for Grand Fenwick whe...,0.773902
2,32671897,"Leaving for vacation, Mickey Mouse and Pluto a...",0.767795
3,878974,What appears to be a large saucer shaped meteo...,0.767222
4,2275930,"When Stanley Putterman installs a brand new, s...",0.764737
5,26163035,The plot centers around restless thirtysomethi...,0.762663
6,1792785,A joint task force operation between the Drug ...,0.762617
7,32712254,"Gabriel Nájera aka ""El Apenas"" has been selli...",0.761622
8,26429769,Terrorists blow up a school bus in the Middle ...,0.761332
9,168498,"Launched in 1977, the Voyager 2 space probe ca...",0.760212
